In [1]:
import sys

sys.path.append("./src")

from pipeline import GCLPipeline

import os
import os.path as osp
import logging
import json
import pandas as pd

import torch
import torch_geometric.transforms as T

from torch.optim import Adam

from tqdm import tqdm

import logging
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s", level=logging.DEBUG, datefmt="%I:%M:%S"
)


DATA_PATH = "./data"
BENCHMARK_PATH = "./benchmark"

OUTPUT_FOLDER = "./output"

In [3]:
# STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "benchmark.json")))
# STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "benchmark-infograph.json")))
# STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "benchmark-dgi.json")))

# STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "augmentation-benchmark-graphcl-infonce.json")))
# STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "augmentation-benchmark-graphcl-jsd.json")))
# STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "augmentation-benchmark-graphcl-barlow.json")))
STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "augmentation-benchmark-graphcl-bootstrap.json")))

# STRATEGIES = json.load(open(osp.join(BENCHMARK_PATH, "loss-benchmark-infograph.json")))

In [4]:
CONFIG = STRATEGIES["config"]
TRY_GPU = CONFIG["try_gpu"]
DEVICE = torch.device("cuda" if torch.cuda.is_available() and TRY_GPU else "cpu")

# Run

In [5]:
outputs = []

for strategy in STRATEGIES["strategies"]:

    logger.info({strategy["_descr"]})

    # Initialization
    pipeline = GCLPipeline.from_strategy(strategy, DEVICE)

    # Data
    dataset, num_features = GCLPipeline.init_dataset(
        strategy["dataset"], DATA_PATH, T.NormalizeFeatures(), strategy["batch_size"]
    )

    # Encoder
    encoder_params = strategy["encoder_params"]
    encoder_params["input_dim"] = num_features
    encoder_model = pipeline.init_encoder(encoder_params, DEVICE)

    # Training
    train_params = strategy["train_params"]
    lr = train_params["learning_rate"]
    epochs = train_params["epochs"]

    optimizer = Adam(encoder_model.parameters(), lr=lr)
    losses = []

    with tqdm(total=epochs, desc="(T)") as pbar:
        for epoch in tqdm(range(1, epochs + 1)):
            loss = pipeline.train_epoch(encoder_model, dataset, optimizer, DEVICE)
            pbar.set_postfix({"loss": loss})

            pbar.update()

    test_result = pipeline.test(encoder_model, dataset, DEVICE)
    print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')

    outputs.append(
        {   
            "Data": strategy["dataset"],
            "Method": strategy["method"],
            "Archi": strategy["architecture"],
            "Mode": strategy["mode"],
            "Obj": strategy["objective"],
            "Neg": strategy["negative"],
            "Aug1": (
                None if strategy["augmentation1"] is None 
                else
                strategy["augmentation1"]["name"]
                if isinstance(strategy["augmentation1"], dict)
                else ', '.join([a["name"] for a in strategy["augmentation1"]])
            ),
            "Aug1Strat": strategy["augmentation1_strat"],
            "Aug2": (
                None if strategy["augmentation1"] is None
                else 
                strategy["augmentation2"]["name"]
                if isinstance(strategy["augmentation2"], dict)
                else ', '.join([a["name"] for a in strategy["augmentation2"]])
            ),
            "Aug2Strat": strategy["augmentation2_strat"],
            "MicroF1": test_result["micro_f1"].round(2),
            "MacroF1": test_result["macro_f1"].round(2),
        }
    )

02:02:15 INFO:{'########## GRAPHCL - G2G - BootstrapLatent - Identity/Identity ##########'}
02:02:15 INFO:CALL GCLPipeline.from_strategy
02:02:15 INFO:	 Method: GraphCL
02:02:15 INFO:	 Augmentation strategy 1: None
02:02:15 INFO:	 Augmentation strategy 2: None
02:02:15 INFO:CALL GCLPipeline.init_augmentations
02:02:15 INFO:	 Strategy: None
02:02:15 INFO:CALL GCLPipeline.init_augmentation
02:02:15 INFO:	 Name: Identity
02:02:15 INFO:CALL GCLPipeline.init_augmentations
02:02:15 INFO:	 Strategy: None
02:02:15 INFO:CALL GCLPipeline.init_augmentation
02:02:15 INFO:	 Name: Identity
02:02:15 INFO:Augmentations: [<GCL.augmentors.identity.Identity object at 0x000002637D5B4150>, <GCL.augmentors.identity.Identity object at 0x000002637D047BD0>]
02:02:15 INFO:CALL GCLPipeline.init_dataset
02:02:15 DEBUG:open file: c:/Users/alexa/Documents/Visual Studio Code/graph-contrastive-learning/data/PTC_MR/processed/data.pt
02:02:15 INFO:	 Number of features: 18
02:02:15 INFO:CALL GCLPipeline.init_encoder
02:

(E): Best test F1Mi=0.5714, F1Ma=0.5714


(T): 100%|██████████| 100/100 [00:07<00:00, 14.09it/s, loss=1.47]
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number o

(E): Best test F1Mi=0.4857, F1Ma=0.4643


(T):   0%|          | 0/100 [00:00<?, ?it/s]c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
(T): 100%|██████████| 100/100 [00:06<00:00, 14.38it/s, loss=-2.99]
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear 

...

(E): Best test F1Mi=0.4571, F1Ma=0.3537


(T): 100%|██████████| 100/100 [00:07<00:00, 13.31it/s, loss=0.0333]
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number

(E): Best test F1Mi=0.4286, F1Ma=0.3881


(T):   0%|          | 0/100 [00:00<?, ?it/s]c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
(T): 100%|██████████| 100/100 [00:07<00:00, 14.14it/s, loss=-1.41]
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear 

(E): Best test F1Mi=0.4571, F1Ma=0.3848


# Result DataFrame

In [6]:
output_df = pd.DataFrame(outputs)
output_df

,Data,Method,Archi,Mode,Obj,Neg,Aug1,Aug1Strat,Aug2,Aug2Strat,MicroF1,MacroF1
0,PTC_MR,GraphCL,DualBranch,G2G,BootstrapLatent,None,Identity,None,Identity,None,0.57,0.57
1,PTC_MR,GraphCL,DualBranch,G2G,BootstrapLatent,None,Identity,None,FeatureMasking,None,0.49,0.46
2,PTC_MR,GraphCL,DualBranch,G2G,BootstrapLatent,None,Identity,None,EdgeRemoving,None,0.37,0.36
3,PTC_MR,GraphCL,DualBranch,G2G,BootstrapLatent,None,Identity,None,RWSampling,None,0.60,0.51
4,PTC_MR,GraphCL,DualBranch,G2G,BootstrapLatent,None,Identity,None,NodeDropping,None,0.63,0.39
5,PTC_MR,GraphCL,DualBranch,G2G,BootstrapLatent,None,FeatureMasking,None,Identity,None,0.57,0.42
6,PTC_MR,GraphCL,DualBranch,G2G,BootstrapLatent,None,FeatureMasking,None,FeatureMasking,None,0.57,0.55
7,PTC_MR,GraphCL,DualBranch,G2G,BootstrapLatent,None,FeatureMasking,None,EdgeRemoving,None,0.29,0.25
8,PTC_MR,GraphCL,DualBranch,G2G,BootstrapLatent,None,FeatureMasking,None,RWSampling,None,0.51,0.49
9,PTC_MR,GraphCL,DualBranch,G2G,BootstrapLatent,None,FeatureMasking,None,NodeDropping,None,0.71,0.63


# Saving

In [7]:
# output_file = "output.csv"

# output_file = "augmentation_benchmark_graphcl_infonce.csv"
# output_file = "augmentation_benchmark_graphcl_jsd.csv"
# output_file = "augmentation_benchmark_graphcl_barlow.csv"
output_file = "augmentation_benchmark_graphcl_bootstrap.csv"
# output_file = "loss_benchmark_infograph.csv"
output_path = osp.join(OUTPUT_FOLDER, output_file)
output_path

'./output\\augmentation_benchmark_graphcl_bootstrap.csv'

In [ ]:
# Deleting output file
# DO NOT REMOVE BEFORE MAKING A COPY
# os.remove(OUTPUT_PATH)

In [8]:
# Adding rows
output_df.to_csv(output_path, mode='a', header=not osp.exists(output_path), index=False)